In [1]:
import pandas as pd
import pathlib
import duckdb
from IPython.display import display, HTML
import unidecode
import textwrap

# --- Parâmetros de Entrada ---
PASTA_PROCESSED = pathlib.Path("dados_catalogo") / "processed"
ARQUIVO_DE_ENTRADA = PASTA_PROCESSED / 'dados_consolidados_filtrados.parquet'
ARQUIVO_DE_SAIDA = PASTA_PROCESSED / 'tabela_dimensao_limpa_para_splink.parquet'

# --- Regras de Negócio Já Definidas ---
# Semanas a serem EXCLUÍDAS da análise de metadados, com base na nossa investigação.
SEMANAS_A_EXCLUIR = ['2024-10-25', '2024-11-21']

print("--- Etapa 3: Análise Exploratória e Preparação da Tabela Dimensão ---")
print("Bloco 1: Configurações carregadas.")

--- Etapa 3: Análise Exploratória e Preparação da Tabela Dimensão ---
Bloco 1: Configurações carregadas.


In [2]:
print("\n" + "="*80)
print(" ANÁLISE EXPLORATÓRIA 1: COMPLETUDE DAS COLUNAS")
print("="*80)
print("Analisando o percentual de preenchimento (não-nulos) de cada coluna...")

try:
    con = duckdb.connect(database=':memory:')
    df_schema = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{str(ARQUIVO_DE_ENTRADA)}');").fetch_df()
    colunas = df_schema['column_name'].tolist()
    
    metricas_completude = []
    for col in colunas:
        query = f"SELECT 100.0 * COUNT(\"{col}\") / COUNT(*) FROM read_parquet('{str(ARQUIVO_DE_ENTRADA)}');"
        pct_preenchimento = con.execute(query).fetchone()[0]
        metricas_completude.append({"Coluna": col, "Preenchimento (%)": pct_preenchimento})
        
    df_completude = pd.DataFrame(metricas_completude).sort_values(by="Preenchimento (%)", ascending=False)

    display(HTML("<h4>Relatório de Completude de Colunas</h4>"))
    display(df_completude.style.format({'Preenchimento (%)': '{:.2f}%'})
            .bar(subset=['Preenchimento (%)'], color='#5DADE2', vmin=0, vmax=100))
    
    con.close()
except Exception as e:
    print(f"ERRO durante a análise de completude: {e}")
    if 'con' in locals(): con.close()


 ANÁLISE EXPLORATÓRIA 1: COMPLETUDE DAS COLUNAS
Analisando o percentual de preenchimento (não-nulos) de cada coluna...


,Coluna,Preenchimento (%)
0,BB_UID,100.00%
1,Platform_Content_ID,100.00%
2,BB_Hash_Unique,100.00%
3,Platform_Name,100.00%
4,Platform_Country,100.00%
5,Package,100.00%
7,Type,100.00%
8,Deeplink,100.00%
10,Is_Original,100.00%
11,Is_Exclusive,100.00%


In [3]:
print("\n" + "="*80)
print(" ANÁLISE EXPLORATÓRIA 2: ESTABILIDADE DAS COLUNAS")
print("="*80)
print("Analisando a variabilidade dos metadados por BB_UID...")

try:
    con = duckdb.connect(database=':memory:')
    
    # Pega todas as colunas exceto as de controle
    colunas_para_analise = [col for col in colunas if col not in ['BB_UID', 'data_ref', 'arquivo_origem']]
    
    # Monta a query para contar valores distintos por UID para cada coluna
    agregacoes_distinct = [f'COUNT(DISTINCT \"{col}\") AS {col}' for col in colunas_para_analise]
    query_estabilidade = f"""
    SELECT
        {', '.join(agregacoes_distinct)}
    FROM
        read_parquet('{str(ARQUIVO_DE_ENTRADA)}')
    GROUP BY
        "BB_UID"
    """
    
    df_contagens = con.execute(query_estabilidade).fetch_df()
    
    # Calcula o percentual de UIDs que possuem mais de 1 valor para cada coluna
    metricas_estabilidade = []
    for col in df_contagens.columns:
        uids_inconsistentes = (df_contagens[col] > 1).sum()
        pct_inconsistente = 100.0 * uids_inconsistentes / len(df_contagens)
        metricas_estabilidade.append({"Coluna": col, "% de UIDs com >1 Valor": pct_inconsistente})
        
    df_estabilidade = pd.DataFrame(metricas_estabilidade).sort_values(by="% de UIDs com >1 Valor", ascending=True)

    display(HTML("<h4>Relatório de Estabilidade de Colunas</h4><p>Quanto menor o percentual, mais estável (confiável) é a coluna.</p>"))
    display(df_estabilidade.style.format({'% de UIDs com >1 Valor': '{:.2f}%'})
            .background_gradient(cmap='Greens_r', subset=['% de UIDs com >1 Valor']))

    con.close()
except Exception as e:
    print(f"ERRO durante a análise de estabilidade: {e}")
    if 'con' in locals(): con.close()


 ANÁLISE EXPLORATÓRIA 2: ESTABILIDADE DAS COLUNAS
Analisando a variabilidade dos metadados por BB_UID...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Coluna,% de UIDs com >1 Valor
40,BB_Launch_Date,0.00%
38,IMDb_ID,0.00%
36,TMDB_ID,0.01%
37,TVDB_ID,0.19%
21,BB_Year,0.73%
9,Is_Original,1.37%
26,BB_Primary_Country,2.61%
39,Season_Numbers,3.37%
8,Seasons,4.55%
27,BB_Countries,5.91%


In [4]:
# ==============================================================================
# BLOCO DE DECISÃO: Sua seleção de colunas foi aplicada.
# ==============================================================================
# A lista abaixo foi criada a partir da sua análise e da imagem fornecida.
# Estas são as colunas que irão compor a nossa Tabela Dimensão.

COLUNAS_PARA_SPLINK = [
    # --- Identificação Essencial ---
    "BB_UID",
    "Type",
    
    # --- Metadados Principais da Obra (FICA) ---
    "BB_Title",
    "BB_Year",
    "BB_Duration",
    "BB_Genres",
    "BB_Directors",
    "BB_Original_Title",
    "IMDb_ID",
    "BB_Countries",
    "BB_Primary_Country",
    "BB_Production_Companies",
    "BB_Primary_Company",
    "TMDB_ID",
    "BB_Cast"
]

print("As seguintes colunas foram selecionadas para a criação da Tabela Dimensão:")
print(textwrap.fill(", ".join(COLUNAS_PARA_SPLINK), 80))

As seguintes colunas foram selecionadas para a criação da Tabela Dimensão:
BB_UID, Type, BB_Title, BB_Year, BB_Duration, BB_Genres, BB_Directors,
BB_Original_Title, IMDb_ID, BB_Countries, BB_Primary_Country,
BB_Production_Companies, BB_Primary_Company, TMDB_ID, BB_Cast


In [5]:

# --- Funções Auxiliares de Limpeza (pré-Splink) ---
def normalizar_string(texto):
    """Aplica limpeza segura em um campo de texto: minúsculas, remove acentos e espaços extras."""
    if texto is None or pd.isna(texto):
        return None
    texto = str(texto).lower()
    texto = unidecode.unidecode(texto)
    return texto.strip()

def normalizar_lista_em_string(texto, separador=','):
    """Normaliza uma string que representa uma lista (ex: elenco, diretores)."""
    if texto is None or pd.isna(texto):
        return None
    nomes = str(texto).split(separador)
    nomes_limpos = sorted([normalizar_string(nome) for nome in nomes if str(nome).strip()])
    return separador.join(nomes_limpos) if nomes_limpos else None

In [6]:
# ==============================================================================
# BLOCO 5: EXECUÇÃO FINAL - Construção da Tabela Dimensão
# ==============================================================================
print("\n" + "="*80)
print(" EXECUÇÃO FINAL: CONSTRUÇÃO DA TABELA DIMENSÃO")
print("="*80)

try:
    con = duckdb.connect(database=':memory:')
    
    # A query agora usa a lista COLUNAS_PARA_SPLINK que você definiu no bloco anterior
    # e a lista de semanas que já havíamos definido
    colunas_selecionadas_sql = ", ".join([f'"{c}"' for c in COLUNAS_PARA_SPLINK + ['data_ref']])
    
    query_crunch_temporal = f"""
    WITH dados_filtrados AS (
        SELECT {colunas_selecionadas_sql}
        FROM read_parquet('{str(ARQUIVO_DE_ENTRADA)}')
        WHERE data_ref NOT IN {tuple(SEMANAS_A_EXCLUIR) if SEMANAS_A_EXCLUIR else '()'}
    ),
    ranking_por_data AS (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY "BB_UID" ORDER BY data_ref DESC) as rn
        FROM dados_filtrados
    )
    SELECT {', '.join([f'"{c}"' for c in COLUNAS_PARA_SPLINK])}
    FROM ranking_por_data
    WHERE rn = 1
    """

    print("\nExecutando o 'crunch temporal' para criar os perfis únicos...")
    df_dim = con.execute(query_crunch_temporal).fetch_df()
    print(f"\n✔ Tabela dimensão criada com sucesso em: '{ARQUIVO_DE_SAIDA}'")
    
    total_perfis = len(df_dim)
    
    print(f"\n--- Resumo da Operação ---")
    print(f"  - Perfis únicos na Tabela Dimensão final: {total_perfis:,}")
    
    display(HTML("<h4>Amostra da Tabela Dimensão Gerada:</h4>"))
    display(df_dim.head(10))
    
    con.close()
    print("\nProcesso concluído.")

except Exception as e:
    print(f"\nOcorreu um erro durante o processo: {e}")
    if 'con' in locals():
        con.close()


 EXECUÇÃO FINAL: CONSTRUÇÃO DA TABELA DIMENSÃO

Executando o 'crunch temporal' para criar os perfis únicos...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✔ Tabela dimensão criada com sucesso em: 'dados_catalogo\processed\tabela_dimensao_limpa_para_splink.parquet'

--- Resumo da Operação ---
  - Perfis únicos na Tabela Dimensão final: 225,117


,BB_UID,Type,BB_Title,BB_Year,BB_Duration,BB_Genres,BB_Directors,BB_Original_Title,IMDb_ID,BB_Countries,BB_Primary_Country,BB_Production_Companies,BB_Primary_Company,TMDB_ID,BB_Cast
0,00b77a05935019a6ad916b8b6d5f5eee,Movie,Carl's Date,2023.0,9.0,"Comedy, Romance, Family, Adventure, Animation,...",Bob Peterson,Carl's Date,tt26736061,US,US,"Pixar Animation Studios, Pixar, Walt Disney Pi...",Pixar Animation Studios,1076364.0,"Edward Asner, Bob Peterson"
1,00bddd10545d483cd77f47c5b5ae47e1,Movie,Live in the USA Uriah Heep,2003.0,83.0,None,None,None,None,None,None,None,None,NaN,None
2,00c539ccc6dc43d4712cda5d01c974f5,Movie,Whatever Happened to Gelitin,2016.0,82.0,"Comedy, Documentary",Angela Christlieb,Whatever Happened to Gelitin,tt5669930,"AT, AUT",AT,None,None,460658.0,"John Waters, Liam Gillick, Salvatore Viviano, ..."
3,00d7c5bdb60afd7bdc08a3cd9eda8372,Series,Malaysia Astro Chinese International Pageant 2019,2019.0,NaN,None,None,None,None,None,None,None,None,NaN,None
4,00fb202cd659102231f50ae6d73dd1b5,Movie,See you in Denver,2006.0,52.0,None,Jan Šikl,None,None,None,None,None,None,NaN,None
5,01010cb1a71b8f2abb7bd1a002a7d8fc,Movie,"Howard Hughes: The Great Aviator - His Life, L...",2004.0,73.0,"Biography, Documentary",Lynn Stevenson,"Howard Hughes: The Great Aviator - His Life, L...",tt0430200,US,US,None,None,301680.0,"Greg Finley, Howard Hughes"
6,01028430437153d18e47298566740802,Movie,School of Intelligent Faith - 27/10/21 - South...,2021.0,68.0,None,Universal Church,None,None,None,None,None,None,NaN,None
7,011fe07e32ad9588a7c5422249d30e77,Movie,Mensagens - Parte 1,2009.0,24.0,Suspense,"Fábio Vieira Rodrigues,Mg",None,None,None,None,None,None,NaN,None
8,013ff47a3ab5e6df384f7a43975910b4,Movie,A Table of Three,2024.0,23.0,"Family, Drama","Luthfi Ihza Mahendra,Irfan Maulana",None,None,None,None,None,None,NaN,None
9,0151b846b0a4b4c78dfe573b9f309ff5,Movie,The King of Comedy,1982.0,109.0,"Comedy, Drama, Crime",Martin Scorsese,The King of Comedy,tt0085794,US,US,"20th Century Fox Film Corporation, Europacorp ...",Twentieth Century Fox Film Corporation,262.0,"Robert De Niro, Jerry Lewis, Diahnne Abbott, S..."



Processo concluído.


In [7]:
# ==============================================================================
# BLOCO 6: LIMPEZA FINAL PRÉ-SPLINK 
# ==============================================================================

# Funções auxiliares de limpeza (mesmas do notebook 4)
def normalizar_string(texto):
    """Aplica limpeza segura: minúsculas, remove acentos e espaços extras."""
    if texto is None or pd.isna(texto):
        return None
    texto = str(texto).lower()
    texto = unidecode.unidecode(texto)
    return texto.strip()

def normalizar_lista_em_string(texto, separador=','):
    """Normaliza uma string que representa lista (ex.: elenco, diretores)."""
    if texto is None or pd.isna(texto):
        return None
    nomes = str(texto).split(separador)
    nomes_limpos = sorted([normalizar_string(n) for n in nomes if str(n).strip()])
    return separador.join(nomes_limpos) if nomes_limpos else None

print("\n" + "="*80)
print(" LIMPEZA FINAL PRÉ-SPLINK (conteúdo do passo 4, agora no passo 3)")
print("="*80)

try:
    # Entrada agora vem diretamente do BLOCO 5 (em memória), sem parquet intermediário
    df = df_dim.copy()

    print("\n--- Etapa de Limpeza Final Pré-Splink ---")
    df_limpo = df.copy()

    # 1) Normalização de campos textuais
    print("\n1. Normalizando campos de texto (títulos, país)...")
    df_limpo['BB_Title'] = df_limpo['BB_Title'].apply(normalizar_string)
    df_limpo['BB_Original_Title'] = df_limpo['BB_Original_Title'].apply(normalizar_string)
    df_limpo['BB_Primary_Country'] = df_limpo['BB_Primary_Country'].apply(normalizar_string)

    # 2) Padronização de listas (elenco e diretores)
    print("2. Padronizando e ordenando listas de elenco e diretores...")
    df_limpo['BB_Cast'] = df_limpo['BB_Cast'].apply(normalizar_lista_em_string)
    df_limpo['BB_Directors'] = df_limpo['BB_Directors'].apply(normalizar_lista_em_string)

    # 3) Ajuste de tipo (ano)
    print("3. Ajustando tipo de dado da coluna 'BB_Year'...")
    df_limpo['BB_Year'] = pd.to_numeric(df_limpo['BB_Year'], errors='coerce').astype('Int64')

    print("\nLimpeza concluída. Verificando uma amostra dos dados transformados:")
    colunas_preview = ['BB_Title', 'BB_Year', 'BB_Cast', 'BB_Directors']
    display(df_limpo[colunas_preview].head())

    # 4) Persistência FINAL (mesmo nome do notebook 4)
    df_limpo.to_parquet(ARQUIVO_DE_SAIDA, index=False)
    print(f"\n✔ Tabela dimensão final e limpa salva em: '{ARQUIVO_DE_SAIDA}'")

except Exception as e:
    print(f"\nOcorreu um erro durante o processo de limpeza: {e}")



 LIMPEZA FINAL PRÉ-SPLINK (conteúdo do passo 4, agora no passo 3)

--- Etapa de Limpeza Final Pré-Splink ---

1. Normalizando campos de texto (títulos, país)...
2. Padronizando e ordenando listas de elenco e diretores...
3. Ajustando tipo de dado da coluna 'BB_Year'...

Limpeza concluída. Verificando uma amostra dos dados transformados:


,BB_Title,BB_Year,BB_Cast,BB_Directors
0,carl's date,2023,"bob peterson,edward asner",bob peterson
1,live in the usa uriah heep,2003,None,None
2,whatever happened to gelitin,2016,"agnes husslein-arco,ali janka,casey spooner,em...",angela christlieb
3,malaysia astro chinese international pageant 2019,2019,None,None
4,see you in denver,2006,None,jan sikl



✔ Tabela dimensão final e limpa salva em: 'dados_catalogo\processed\tabela_dimensao_limpa_para_splink.parquet'
